# spaCy similarity - second approach

In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet')
nltk.download('stopwords')
!python -m spacy download en_core_web_md 
import spacy


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2022-11-01 04:38:44.374062: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [33]:
nlp = spacy.load('en_core_web_sm')

In [34]:
wine_df = pd.read_csv('wine.csv', index_col=0).reset_index(drop=True)
wine_df.shape

(3050, 3)

In [35]:
wine_df.head()

,product_name,reviews,rating
0,19 Crimes Red Blend 2020,Hard to fault this wine at its price. Vanilla ...,4.2
1,19 Crimes Red Blend 2020,A treasure for the price. Especially with a cl...,4.5
2,19 Crimes Red Blend 2020,"Lovely bouquet, first off vanilla balanced by ...",4.0
3,19 Crimes Red Blend 2020,"Awesome red blend! Well balanced, complex, fru...",4.0
4,19 Crimes Red Blend 2020,Another wine from 19 crimes what I tasted. Aga...,4.8


In [36]:
# Pre-process message texts
wine_df['reviews'] = wine_df['reviews'].astype(str)
wine_df['reviews'] = wine_df['reviews'].apply(lambda x: x.lower())

# Keep only words with letters
wine_df['reviews'] = wine_df['reviews'].apply(lambda x: ' '.join(re.findall('([A-z]+)', x)))

In [37]:
tokenizer = RegexpTokenizer(r'\w+')
wine_df['Words'] = wine_df['reviews'].apply(lambda x: tokenizer.tokenize(x))

In [38]:
stopwords_1 = stopwords.words('english')
wine_df['product_review clean'] = wine_df['Words'].apply(lambda x: ' '.join([word for word in x if word not in stopwords_1]))

In [39]:
# Drop reviews where cleaned words is empty
wine_df = wine_df[wine_df['product_review clean'].apply(lambda x: len(x) > 0)]
wine_df = wine_df.reset_index()
wine_df.shape

(3050, 6)

In [40]:
nlp = spacy.load('en_core_web_md')

In [41]:
attr_list = ['tannin']

In [42]:
wine_df['Spacy Similarity'] = wine_df['product_review clean'].apply(lambda x: nlp(' '.join(attr_list)).similarity(nlp(x)))
wine_df['Spacy Similarity']

0       0.569865
1       0.409652
2       0.647282
3       0.517720
4       0.607864
          ...   
3045    0.517117
3046    0.614961
3047    0.283793
3048    0.611677
3049    0.171382
Name: Spacy Similarity, Length: 3050, dtype: float64